In [ ]:
import pandas as pd
import matplotlib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import classification_report, accuracy_score
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import tensorflow as tf
import keras

In [ ]:
train_sample_metadata = pd.read_json('Data/train_sample_videos/metadata.json').T
train_sample_metadata.head()

In [ ]:
IMAGE_SIZE = (124,124)

In [ ]:
def load_data(dir):
    categories = ['fake','real']
    images = []
    labels = []
    for i,category in enumerate(categories):
        path = os.path.join(dir,category)
        label = i
        for folder in os.listdir(path):
            for file in os.listdir(os.path.join(path,folder)):
                img_path = os.path.join(os.path.join(path,folder),file)
                image = cv2.imread(img_path,0)
                image = cv2.resize(image, IMAGE_SIZE)
                images.append(image)
                labels.append(label)
    images = np.array(images, dtype='float32')
    labels = np.array(labels, dtype= 'int32')
    return (images,labels)

In [ ]:
dir = '/Users/audunlien/Documents/DAT550/DAT550-prosjekt/Data/train/train'
(train_x,train_y) = load_data(dir)

In [ ]:
dir = '/Users/audunlien/Documents/DAT550/DAT550-prosjekt/Data/test_set/test_set'
(test_x,test_y) = load_data(dir)

In [ ]:
from sklearn.utils import shuffle
train_x, train_y = shuffle(train_x, train_y, random_state=0)

In [ ]:
from sklearn.utils import shuffle
test_x, test_y = shuffle(test_x, test_y, random_state=0)

In [26]:
rnn = Sequential()
rnn.add(LSTM(128, input_shape=(124,124), activation='relu', return_sequences=True))
rnn.add(Dropout(0.2))

rnn.add(LSTM(64, activation='relu',return_sequences=True))
rnn.add(Dropout(0.2))
rnn.add(LSTM(64, activation='relu'))
rnn.add(Dropout(0.2))
rnn.add(Dense(32, activation='softmax'))
rnn.add(Dropout(0.2))

rnn.add(Dense(1, activation='sigmoid'))
#softmax

In [27]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6)
rnn.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=["accuracy"],
)


In [ ]:
epochs = 30
batches = 128

In [ ]:
total_train_batches = len(train_x)//batches
total_val_batches = len(test_x)//batches

In [ ]:
with tf.device('/cpu:0'):
    history = rnn.fit(train_x,train_y,
            epochs=epochs,
            steps_per_epoch= total_train_batches,
            validation_data=(test_x,test_y),
            validation_steps=total_val_batches,
            batch_size= batches,
            )

In [ ]:
rnn.save('my_model_2.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
print("[INFO] evaluating network...")
predIdxs = rnn.predict(test_x, batch_size=128)
predIdx = np.round(predIdxs).astype(int)
# show a nicely formatted classification report
print(classification_report(test_y, predIdx,target_names=['fake','real']))



In [ ]:
accuracy_score(test_y, predIdx) 

In [ ]:
import keras
from matplotlib import pyplot as plt
%matplotlib inline
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()